In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
train.convert_dtypes().dtypes
test.convert_dtypes().dtypes

PassengerId      Int64
Pclass           Int64
Name            string
Sex             string
Age            Float64
SibSp            Int64
Parch            Int64
Ticket          string
Fare           Float64
Cabin           string
Embarked        string
dtype: object

In [4]:
train.loc[train['Sex'] == 'male', 'Sex_num'] = 0
train.loc[train['Sex'] == 'female', 'Sex_num'] = 1

test.loc[test['Sex'] == 'male', 'Sex_num'] = 0
test.loc[test['Sex'] == 'female', 'Sex_num'] = 1

In [5]:
avg_age = train.pivot_table(columns=['Sex','Pclass'], values='Age', aggfunc='mean').mean()

avg_age = avg_age.reset_index()
avg_age.columns = ['Sex', 'Pclass', 'Value']

avg_age

,Sex,Pclass,Value
0,female,1,34.611765
1,female,2,28.722973
2,female,3,21.750000
3,male,1,41.281386
4,male,2,30.740707
5,male,3,26.507589


In [6]:
train = train.merge(avg_age, left_on=['Sex','Pclass'], right_on=['Sex','Pclass'], how='left')

train['Age'].fillna(train['Value'], inplace=True)

test = test.merge(avg_age, left_on=['Sex','Pclass'], right_on=['Sex','Pclass'], how='left')

test['Age'].fillna(train['Value'], inplace=True)

In [7]:
train.set_index('PassengerId', inplace=True)
train.drop(columns=['Name','Ticket','Cabin','Fare','Embarked','Sex','Value'], inplace=True)
train.rename(columns={'Sex_num':'Sex'}, inplace=True)

test.set_index('PassengerId', inplace=True)
test.drop(columns=['Name','Ticket','Cabin','Fare','Embarked','Sex','Value'], inplace=True)
test.rename(columns={'Sex_num':'Sex'}, inplace=True)

In [8]:
target = pd.DataFrame(train, columns=['Survived'])
print(target.head())

train.drop(columns=['Survived'], axis=1, inplace=True)
print(train.head())

             Survived
PassengerId          
1                   0
2                   1
3                   1
4                   1
5                   0
             Pclass   Age  SibSp  Parch  Sex
PassengerId                                 
1                 3  22.0      1      0  0.0
2                 1  38.0      1      0  1.0
3                 3  26.0      0      0  1.0
4                 1  35.0      1      0  1.0
5                 3  35.0      0      0  0.0


In [9]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier

In [10]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(train, target)

prediction = knn.predict(test)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:200: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [11]:
train['Survived'] = target
test['Survived'] = prediction

chart_export = pd.concat([train, test])
chart_export.to_csv('chart_export.csv')

submit = test.drop(columns=['Pclass','Age','SibSp','Parch','Sex']).reset_index()
submit.to_csv('submission.csv',index=False)